In [9]:
# Magic! (don't worry about this)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
# Let use use some useful things. See lib.py for the source code.
from lib import *
import math
import numpy as np

In [48]:
# load the data
data_store = Data()
train = data_store.get_train()

In [53]:
class KNearestNeighbors:
    
    def train(self, data):
        self.train = data
        
    def classify(self, data, k, distance_metric):
        labelled = []
        for d in data:
            guess = self.classify_point(d, k, distance_metric)
            labelled.append((d, guess))
            if guess == -100:
                print 'nanness!'
                break
        return labelled
        
    def classify_point(self, test_point, k, distance_metric):
        # find the k nearest neighbors
        distances = []
        for train_point in self.train:
            distance = distance_metric(test_point, train_point)
            distances.append((distance, train_point))
            if math.isnan(distance):
                return -100
        distances.sort()
        closest = distances[:k]
        # take a majority vote
        votes = {}
        for close in closest:
            label = close[1].get_label()
            if not label in votes:
                votes[label] = 0
            votes[label] += 1
        # choose the label with the most votes
        max_votes = 0
        max_label = 0
        for v in votes:
            if votes[v] > max_votes:
                max_label = v
                max_votes = votes[v]
        return max_label

In [67]:
def euclidean_distance(s1, s2):
    distance = float(0)
    m1 = s1.get_microarray()
    m2 = s2.get_microarray()
    for i in range(len(m1)):
        distance += float(pow((m1[i] - m2[i]), 2))
    return math.sqrt(distance)

def euclidean_efficient(s1, s2):
    m1 = np.array(s1.get_microarray())
    m2 = np.array(s2.get_microarray())
    return np.linalg.norm(np.asarray(m1, dtype=np.float64) - np.asarray(m2, dtype=np.float64))
    

In [71]:
knn = KNearestNeighbors()
knn.train(train)

labelled = knn.classify(data_store.get_test(), 3, euclidean_efficient)

In [72]:
#print labelled

In [73]:
analyze_results(labelled)

Accuracy: 62.07%


In [58]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)
train_matrix = []
train_labels = []
for s in train:
    train_matrix.append(s.get_microarray())
    train_labels.append(s.get_label())
clf.fit(train_matrix, train_labels)

def classify(clf, points):
    labelled = []
    for p in points:
        label = clf.predict(p.get_microarray())[0]
        labelled.append((p, label))
    return labelled

In [59]:
clf_labelled = classify(clf, data_store.get_test())
analyze_results(clf_labelled)

Accuracy: 62.07%
